In [91]:
import numpy as np
from scipy.stats import norm
from scipy.spatial import Delaunay
import open3d as o3d
import openstl

In [92]:
import math

# set specs

resolution = 6

mu, sigma = 0, 1

side_length = 5 * sigma
side_height = side_length / 20
print(side_length)

5


In [93]:
x_max = side_length / 2
x_min = -x_max
z_max = side_length / 2
z_min = -z_max
y_max = side_height
y_min = 0

n_side = int(side_length*resolution)
n_side = 30

In [94]:
# A SIDE
x = np.linspace(x_min, x_max, n_side)
y = np.linspace(y_min, y_max, 5)
# print(y)
X, Y = np.meshgrid(x, y)
Z = np.full(shape=(len(x) * len(y)), fill_value=z_max)
a_verts = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

In [95]:
# B SIDE
x = np.linspace(x_min, x_max, n_side)
y = np.linspace(y_min, y_max, 5)
# print(y)
X, Y = np.meshgrid(x, y)
Z = np.full(shape=(len(x) * len(y)), fill_value=z_min)
b_verts = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T


In [96]:
# C SIDE
z = np.linspace(z_min, z_max, n_side)
y = np.linspace(y_min, y_max, 5)
# print(y)
Z, Y = np.meshgrid(z, y)
X = np.full(shape=(len(z) * len(y)), fill_value=x_min)
c_verts = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

In [97]:
# D SIDE
z = np.linspace(z_min, z_max, n_side)
y = np.linspace(y_min, y_max, 5)
# print(y)
Z, Y = np.meshgrid(z, y)
X = np.full(shape=(len(z) * len(y)), fill_value=x_max)
d_verts = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

In [98]:
# render_pcds([a_verts, b_verts, c_verts, d_verts])

In [99]:
def gen_side_mesh(verts, side: str):
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(verts)
    simplices = None
    if side == 'a' or side == 'b':
        simplices = Delaunay(verts[:, :2]).simplices
        # mesh.triangles = o3d.utility.Vector3iVector(Delaunay(verts[:, :2]).simplices)
    elif side == 'c' or side == 'd':
        simplices = Delaunay(verts[:, [1, 2]]).simplices
        # mesh.triangles = o3d.utility.Vector3iVector(Delaunay(verts[:, [1, 2]]).simplices)

    if side == 'b' or side == 'c':
        flipped_simplices = []
        for tri in simplices:
            flipped_simplices.append(np.flip(tri))
        simplices = flipped_simplices

    mesh.triangles = o3d.utility.Vector3iVector(simplices)
    mesh.compute_vertex_normals()
    return mesh

In [100]:
a_side = gen_side_mesh(a_verts, 'a')
a_side.paint_uniform_color([1, 0, 0])
b_side = gen_side_mesh(b_verts, 'b')
b_side.paint_uniform_color([0, 1, 0])
c_side = gen_side_mesh(c_verts, 'c')
c_side.paint_uniform_color([0, 0, 1])
d_side = gen_side_mesh(d_verts, 'd')
d_side.paint_uniform_color([1, 1, 0])
meshes = [a_side, b_side, c_side, d_side]
# o3d.visualization.draw_geometries(meshes)

In [101]:
def gen_top_mesh(verts):
    mesh = o3d.geometry.TriangleMesh()

    mesh.vertices = o3d.utility.Vector3dVector(verts)
    simplices = []
    for tri in Delaunay(verts[:, [0, 2]]).simplices:
        simplices.append(np.flip(tri))
    mesh.triangles = o3d.utility.Vector3iVector(simplices)
    mesh.compute_vertex_normals()
    return mesh


In [102]:
def render_pcds(meshes):
    center = o3d.geometry.PointCloud()
    center.points = o3d.utility.Vector3dVector(np.array([[0,0,0]]))
    pcds = []
    pcds.append(center)
    for mesh in meshes:
        pcd = o3d.geometry.PointCloud()
        pcd.points = mesh.vertices
        pcds.append(pcd)
    o3d.visualization.draw_geometries(pcds)

In [103]:
# TOP SURFACE
x = np.linspace(x_min, x_max, n_side)
z = np.linspace(z_min, z_max, n_side)
# print(y)
X, Z = np.meshgrid(x, Z)
Y = norm.pdf(X, mu, sigma) * norm.pdf(Z, mu, sigma) * 10

top_verts = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T

# zeroize edges
for coord in top_verts:
    if coord[0] == x_min or coord[0] == x_max or coord[2] == z_min or coord[2] == z_max:
        coord[1] = 0

top_verts[:,1] = top_verts[:,1] + side_height

top_mesh = gen_top_mesh(top_verts)
meshes.append(top_mesh)


In [108]:
render_pcds(meshes)
# o3d.visualization.draw_geometries(meshes)
mesh = a_side + b_side + c_side + d_side + top_mesh
print(mesh.is_watertight())
o3d.visualization.draw_geometries([mesh])
o3d.io.write_triangle_mesh('..\\out\\mesh.stl', mesh)

False


True

In [90]:
# o3d.visualization.draw_geometries([top_mesh])